<h1 style = "text-align: center;" markdown = "1"> DIAKITE MAMADOU YOUSSOUF </h1>
<h3 style = "text-align: center;" markdown = "2"> Master 2 BIG DATA -- DATA SCIENCE </h3>

<h3 style = "text-align: center;" markdown = "2"> SPARK </h3>

In [1]:
import os

try:
    os.remove("metastore_db/db.lck")
    os.remove("metastore_db/dbex.lck")
except:
    pass


from pyspark.sql import SparkSession

def build_spark_session(app_name, memory='4g', executors=4):
    return SparkSession.builder\
                      .appName(app_name)\
                      .config('spark.executor.memory', memory)\
                      .config('spark.executor.instances', executors)\
                      .getOrCreate()

spark_session = build_spark_session(app_name='ok-google')

from pyspark.sql import functions as f



1 - joindre les dataframe sur la clé ID_CLIENT en concervant tous les clients du périmètre.
2 - compter le nombre de ID_CLIENT et vérifier qu'il correspond aux nombre d'ID_CLIENT dans la variable perimètre.
3 - Caster les variables continues en double et sauvergarder alors le df obtenu dans le repertoire data sur le cluster.
4 - Pour les variables catégorielles, créer une nouvelle variable qui prend la modalité de la variable courante si elle existe et "NA" sinon.
5- Verifier la cohérence des variables continue. Par exemple pour une variable comme age mettre à -1 tous les ages <0 ou>120ans

In [2]:
# Loading files (reading files)

perimetre = spark_session.read.csv("data_clients/sample_perimetre.csv", header=True)
histo_client_raw = spark_session.read.csv("data_clients/sample_histo_client.csv", header=True)
histo_train_raw = spark_session.read.csv("data_clients/sample_histo_train.csv", header=True)
histo_lowcost_raw = spark_session.read.csv("data_clients/sample_histo_lowcost.csv", header=True)
visites_raw = spark_session.read.csv("data_clients/sample_visites.csv", header=True)

In [3]:
# Identify continuous variables

print(perimetre.printSchema())
print(histo_client_raw.printSchema())
print(histo_lowcost_raw.printSchema())
print(visites_raw.printSchema())


root
 |-- ID_CLIENT: string (nullable = true)

None
root
 |-- ID_CLIENT: string (nullable = true)
 |-- anciennete: string (nullable = true)
 |-- recence_cmd: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- LBL_STATUT_CLT: string (nullable = true)
 |-- LBL_GEO_AIR: string (nullable = true)
 |-- LBL_GRP_SEGMENT_NL: string (nullable = true)
 |-- LBL_SEG_COMPORTEMENTAL: string (nullable = true)
 |-- LBL_GEO_TRAIN: string (nullable = true)
 |-- LBL_SEGMENT_ANTICIPATION: string (nullable = true)
 |-- FLG_CMD_CARTE_1225: string (nullable = true)

None
root
 |-- ID_CLIENT: string (nullable = true)
 |-- flg_cmd_lowcost: string (nullable = true)
 |-- flg_track_nl_lowcost: string (nullable = true)
 |-- flg_track_nl: string (nullable = true)

None
root
 |-- ID_CLIENT: string (nullable = true)
 |-- days_since_last_visit: string (nullable = true)
 |-- tx_conversion: string (nullable = true)

None


#### Ecrire une fonction pour transformer 
#### les features quantitatives ("anciennete", "recence_cmd", "AGE", etc..) en float

In [7]:
# Transform quantitatives features in double 

client_cols_to_keep = ["ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

def cast_columns_of_df(df, cols_to_cast, col_to_keep, cast_type='double'):
    """cast continuous columns into double since all columns are """
    return df.select(col_to_keep + [(df[feature].cast(cast_type))
                    for feature in cols_to_cast if 'ID_CLIENT' not in feature])

histo_train = cast_columns_of_df(histo_train_raw, histo_train_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

histo_lowcost = cast_columns_of_df(histo_lowcost_raw, histo_lowcost_raw.columns,
                                 ["ID_CLIENT"], cast_type='double')

visites = cast_columns_of_df(visites_raw, visites_raw.columns,
                             ["ID_CLIENT"], cast_type='double')

histo_client = cast_columns_of_df(histo_client_raw,
                                  ["anciennete", "recence_cmd", "AGE"],
                                  client_cols_to_keep,
                                 cast_type='double')


#### Join part

In [8]:
# Firsly we count the number of row of perimetre
perimetre.count()

1084217

In [9]:
# We join sequentially file perimetre with the others files 
per11 = perimetre.join(histo_client, on = ['ID_CLIENT'], how = 'left_outer')

In [10]:
per12 = per11.join(histo_lowcost, on = ['ID_CLIENT'], how = 'left_outer') 

In [11]:
per13 = per12.join(histo_train, on = ['ID_CLIENT'], how = 'left_outer')

In [12]:
per14 = per13.join(visites, on = ['ID_CLIENT'], how = 'left_outer')

In [13]:
# We check the number of row of our final dataset
# It equals to the number of row of perimetre
per14.count()

1084217

#### Quelles sont les differentes modalites de la feature LBL_STATUT_CLT

In [14]:
# Differentes modalités de la variable LBL_STATUT_CLT
per14.select('LBL_STATUT_CLT').distinct().toPandas()

,LBL_STATUT_CLT
0,Moyen moins
1,Non present dans la base a cette date
2,Nouveau prospect
3,Prospect
4,Tres petit
5,None
6,Petit
7,Inactif
8,Nouveau actif
9,Grand


#### Quelles sont les features avec valeurs manquantes

In [15]:
# Features with missing data

from pyspark.sql.functions import col,sum
# We can count the missing values by summing the boolean output of the isNull() method

def check_missing_values(df) :

    return df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).toPandas()


In [16]:
check_missing_values(per14)

,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR,LBL_SEG_COMPORTEMENTAL,LBL_GEO_TRAIN,LBL_GRP_SEGMENT_NL,LBL_SEGMENT_ANTICIPATION,FLG_CMD_CARTE_1225,anciennete,recence_cmd,...,nb_od,mean_nb_passagers,mean_duree_voyage,mean_mt_voyage,mean_tarif_loisir,mean_classe_1,mean_pointe,mean_depart_we,days_since_last_visit,tx_conversion
0,0,78998,162977,155160,163010,79522,157822,10283,55,1484,...,49927,49927,49927,49927,62370,49927,49927,49927,220485,220485


We can see that almost all variables have missing except "ID_CLIENT" and "LBL_STATUT_CLT".

In [17]:
# Number of quantitatives and qualitatives features
per14.dtypes

[('ID_CLIENT', 'string'),
 ('LBL_STATUT_CLT', 'string'),
 ('LBL_GEO_AIR', 'string'),
 ('LBL_SEG_COMPORTEMENTAL', 'string'),
 ('LBL_GEO_TRAIN', 'string'),
 ('LBL_GRP_SEGMENT_NL', 'string'),
 ('LBL_SEGMENT_ANTICIPATION', 'string'),
 ('FLG_CMD_CARTE_1225', 'string'),
 ('anciennete', 'double'),
 ('recence_cmd', 'double'),
 ('AGE', 'double'),
 ('flg_cmd_lowcost', 'double'),
 ('flg_track_nl_lowcost', 'double'),
 ('flg_track_nl', 'double'),
 ('nb_od', 'double'),
 ('mean_nb_passagers', 'double'),
 ('mean_duree_voyage', 'double'),
 ('mean_mt_voyage', 'double'),
 ('mean_tarif_loisir', 'double'),
 ('mean_classe_1', 'double'),
 ('mean_pointe', 'double'),
 ('mean_depart_we', 'double'),
 ('days_since_last_visit', 'double'),
 ('tx_conversion', 'double')]

We have 8 quantitatives features and 16 qualitatives features 

#### Remplacer les valeurs manquantes par -1 pour toutes les features qualitatives et par la moyenne
#### pour toutes les features quantitatives

In [18]:
# Here we select continuous variables

def select_continous_features(df) :
    LL = []
    for i in df.dtypes :
        if i[1] == 'double' :
            LL.append(i[0])
    return LL

In [19]:
continuous_columns = select_continous_features(per14)

In [20]:
# We remove 'flg_cmd_lowcost' from our list our continuous values
del(continuous_columns[3]) 

In [21]:
continuous_columns

['anciennete',
 'recence_cmd',
 'AGE',
 'flg_track_nl_lowcost',
 'flg_track_nl',
 'nb_od',
 'mean_nb_passagers',
 'mean_duree_voyage',
 'mean_mt_voyage',
 'mean_tarif_loisir',
 'mean_classe_1',
 'mean_pointe',
 'mean_depart_we',
 'days_since_last_visit',
 'tx_conversion']

In [22]:
# Input missing value 
# for qualitative variable input with -1
# for quantitative with mean 
from pyspark.sql.functions import col,sum

def missing_values_input(df):
    features_quali = ['flg_cmd_lowcost', "ID_CLIENT", 'LBL_STATUT_CLT','LBL_GEO_AIR',
            'LBL_SEG_COMPORTEMENTAL','LBL_GEO_TRAIN','LBL_GRP_SEGMENT_NL',
            'LBL_SEGMENT_ANTICIPATION','FLG_CMD_CARTE_1225']

    df2 = df.select([f.when(df[feature].isNotNull(), df[feature])\
                .otherwise('-1').alias(feature) for feature in features_quali]\
               + [f.when(df[feature].isNotNull(), df[feature])\
                 .otherwise(df.agg({feature :'mean'}).toPandas().values[0][0]).alias(feature)
                  for feature in continuous_columns])

    return df2

In [23]:
per17 = missing_values_input(per14)

In [24]:
# Now per17 is our new dataset
per17.toPandas().iloc[0:2,0:4]

,flg_cmd_lowcost,ID_CLIENT,LBL_STATUT_CLT,LBL_GEO_AIR
0,-1,000843db32fbaecfbb047ca0bb04b1f9f4d9425a,Grand,Aéroports de Paris Orly
1,-1,001338752ea32d9de129c8f8bdf3e2224cf0bd71,Grand,Aéroport de Marseille Provence (MRS)


In [25]:
per17.printSchema()

root
 |-- flg_cmd_lowcost: string (nullable = true)
 |-- ID_CLIENT: string (nullable = true)
 |-- LBL_STATUT_CLT: string (nullable = true)
 |-- LBL_GEO_AIR: string (nullable = true)
 |-- LBL_SEG_COMPORTEMENTAL: string (nullable = true)
 |-- LBL_GEO_TRAIN: string (nullable = true)
 |-- LBL_GRP_SEGMENT_NL: string (nullable = true)
 |-- LBL_SEGMENT_ANTICIPATION: string (nullable = true)
 |-- FLG_CMD_CARTE_1225: string (nullable = true)
 |-- anciennete: double (nullable = true)
 |-- recence_cmd: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- flg_track_nl_lowcost: double (nullable = true)
 |-- flg_track_nl: double (nullable = true)
 |-- nb_od: double (nullable = true)
 |-- mean_nb_passagers: double (nullable = true)
 |-- mean_duree_voyage: double (nullable = true)
 |-- mean_mt_voyage: double (nullable = true)
 |-- mean_tarif_loisir: double (nullable = true)
 |-- mean_classe_1: double (nullable = true)
 |-- mean_pointe: double (nullable = true)
 |-- mean_depart_we: double (

#### Quelles sont les differentes valeurs de notre label : flg_cmd_lowcost

In [26]:
per17.select('flg_cmd_lowcost').distinct().toPandas()

,flg_cmd_lowcost
0,1.0
1,-1


##### Features engineering et modélisation

In [27]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression


In [28]:
def input_df(df):
    ds = df.select('ID_CLIENT',
    f.when(df.LBL_GEO_TRAIN.isin(['Toulouse', 'Lille', 'Dijon',
                                  'Lyon', 'Marseille', 'Paris',
                                  'Nice', 'Limoges','Rouen','Rennes',
                                  'Montpellier', 'Bordeaux', 'Metz',
                                  'Strasbourg']), df.LBL_GEO_TRAIN)\
               .otherwise('na').alias('geo_train'),
    f.when(df.LBL_GEO_AIR.isin(['Aéroports de Paris Orly',
                                'Aéroport de Bâle-Mulhouse / Bassel',
                                'Aéroport Lille Lesquin', 'Aéroport de Rennes',
                                'Aéroport de Nantes Atlantique',
                                'Aéroport de Marseille Provence  (MRS)', 
                                'Aéroport de Bordeaux Mérignac',
                                'Aéroports de Paris Roissy-Charles-de Gaulle', 
                                "Aéroport de Nice Côte d'Azur",
                                'Aéroport de Strasbourg',
                                'Aéroport de Lyon - Saint Exupéry', 
                                'Aéroport de Toulouse Blagnac']), df.LBL_GEO_AIR)\
               .otherwise('na').alias('geo_air'),
    f.when(df.FLG_CMD_CARTE_1225 == '1', '1')\
                   .otherwise('0').alias('cc_jeunes'),
    f.when(df.LBL_STATUT_CLT.isin(['Tres grand', 'Nouveau actif',
                                   'Moyen moins', ' Prospect', ' Petit',
                                   'Inactif', 'Tres petit',
                                   'Nouveau prospect', 'Moyen plus',
                                   'Grand']), df.LBL_STATUT_CLT)\
                   .otherwise('na').alias('segt_rfm'),
    f.when(df.LBL_SEGMENT_ANTICIPATION.isin(['Peu Anticipateur', 'Tres Anticipateur',
                                             'Anticipateur', 'Mixte', 'Non Anticipateur',
                                             'Non Defini']), df.LBL_SEGMENT_ANTICIPATION)\
                   .otherwise('na').alias('segt_anticipation'),
    f.when(df.LBL_SEG_COMPORTEMENTAL.isin(['Mono-commande',
                                           'Comportement Pro',
                                           'Exclusifs Agence', 
                                           'Anticipateurs Methodiques',
                                           'Chasseurs Bons Plans', 
                                           'Rythmes scolaires', 'Nouveaux',
                                           'Sans contraintes']),
           df.LBL_SEG_COMPORTEMENTAL).otherwise('na').alias('segt_comportemental'), 
    f.when(df.LBL_GRP_SEGMENT_NL.isin(['Endormi', 'Spectateur', 'Acteur',
                                       'Eteint', 'Non defini']),
           df.LBL_GRP_SEGMENT_NL).otherwise('na').alias('segt_nl'),
    f.when(((df.AGE > 0) & (df.AGE < 100)), df.AGE)\
                   .otherwise(-1).alias('age'),
    f.when(df.recence_cmd >= 0, df.recence_cmd)\
                   .otherwise(-1).alias('recence_cmd'),
    f.when(((df.mean_duree_voyage > 0) & (df.mean_duree_voyage < 750)),
           df.mean_duree_voyage).otherwise(-1).alias('mean_duree_voyage'),
    f.when(df.days_since_last_visit >= 0, df.days_since_last_visit)\
                   .otherwise(-1).alias('recence_visite'),
    f.when(df.mean_mt_voyage > 0, df.mean_mt_voyage)\
                   .otherwise(-1).alias('mean_mt_voyage'),
    f.when(df.anciennete >= 0, df.anciennete)\
                   .otherwise(-1).alias('anciennete'),
    f.when(df.nb_od > 0, df.nb_od)\
                   .otherwise(-1).alias('nb_od'),
    f.when(df.mean_nb_passagers > 0, df.mean_nb_passagers)\
                   .otherwise(-1).alias('mean_nb_passagers'),
    f.when(df.mean_tarif_loisir >= 0, df.mean_tarif_loisir)\
                   .otherwise(-1).alias('mean_tarif_loisir'),
    f.when(df.mean_classe_1 >= 0, df.mean_classe_1)\
                   .otherwise(-1).alias('mean_classe_1'),
    f.when(df.mean_pointe >= 0, df.mean_pointe)\
                   .otherwise(-1).alias('mean_pointe'),
    f.when(df.mean_depart_we >= 0, df.mean_depart_we)\
                   .otherwise(-1).alias('mean_depart_we'),
    f.when(df.tx_conversion >= 0, df.tx_conversion)\
                   .otherwise(-1).alias('tx_conversion'),
    f.when(df.flg_cmd_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_cmd_lowcost'),
    f.when(df.flg_track_nl_lowcost == 1, '1')\
                   .otherwise('0').alias('flg_track_nl_lowcost'), 
    f.when(df.flg_track_nl == 1, '1')\
                   .otherwise('0').alias('flg_track_nl'))
    
    return ds


In [29]:
df = input_df(per17)

In [30]:

def preprocessed_df(df, label="flg_cmd_lowcostIndex"):
    max_values_to_define_str_cols = 10
    id_col = 'ID_CLIENT'
    
    dty = dict(df.dtypes)
    str_cols = [k for k, v in dty.items() if v == 'string']
    str_cols.remove(id_col)
    
    for c in str_cols:
        stringIndexer = StringIndexer(inputCol=c, outputCol=c+"Index")
        model_str = stringIndexer.fit(df)
        df = model_str.transform(df).drop(c)

    input_cols = df.columns
    input_cols.remove(id_col)
    input_cols.remove(label)
    
    assembler = VectorAssembler(inputCols=input_cols,
                            outputCol="features")
    df = assembler.transform(df)
    
    featureIndexer = VectorIndexer(inputCol="features", 
                   outputCol="indexedFeatures", 
                   maxCategories=max_values_to_define_str_cols).fit(df)
    return featureIndexer.transform(df), df


data, dff = preprocessed_df(df)



In [31]:
# We print the first row of our data

data.head(1)

[Row(ID_CLIENT='000843db32fbaecfbb047ca0bb04b1f9f4d9425a', age=36.77269796022761, recence_cmd=36.0, mean_duree_voyage=274.6666666666667, recence_visite=8.0, mean_mt_voyage=58.666666666666664, anciennete=1550.0, nb_od=1.0, mean_nb_passagers=1.0, mean_tarif_loisir=0.0, mean_classe_1=0.0, mean_pointe=0.0, mean_depart_we=0.0, tx_conversion=0.1111111111111111, geo_trainIndex=0.0, geo_airIndex=2.0, cc_jeunesIndex=0.0, segt_rfmIndex=2.0, segt_anticipationIndex=4.0, segt_comportementalIndex=6.0, segt_nlIndex=1.0, flg_cmd_lowcostIndex=0.0, flg_track_nl_lowcostIndex=0.0, flg_track_nlIndex=0.0, features=DenseVector([36.7727, 36.0, 274.6667, 8.0, 58.6667, 1550.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1111, 0.0, 2.0, 0.0, 2.0, 4.0, 6.0, 1.0, 0.0, 0.0]), indexedFeatures=DenseVector([36.7727, 36.0, 274.6667, 8.0, 58.6667, 1550.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1111, 0.0, 2.0, 0.0, 2.0, 4.0, 6.0, 1.0, 0.0, 0.0]))]

#### Prelever un sample de data pour notre modelisation

In [32]:
sample_data = data.sample(False, 0.01, seed = 0)

In [33]:
sample_data.count()

10954

#### Logistic regression

#### Quelle est le label est renseigne pour la modelisation?

Le label est "flg_cmd_lowcostIndex"

In [34]:
# Split our data
train, test = sample_data.randomSplit([0.8, 0.2], seed=12345)

In [35]:
# We start performing our logistic regression 

lr = LogisticRegression(labelCol="flg_cmd_lowcostIndex", 
                        featuresCol="indexedFeatures",elasticNetParam=0.5)


In [36]:
# We fit the logistic model
model = lr.fit(train)

#### Faire une fonction qui retourne score, recall, precision, AUC sous forme de dictionnaire

We make our function , it returns accuracy, precision, recall and AUC.

In [47]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

def compute_model(method,label, data) :
    
    skill = method 
    
    train, test = data.randomSplit([0.8, 0.2], seed=12345)
    
    model = skill.fit(train)
    
    predict_test = model.transform(test)
    
    results = predict_test.select(['prediction', label])
    
    predictionAndLabels = results.rdd

    metrics = MulticlassMetrics(predictionAndLabels)
    
    cm = metrics.confusionMatrix().toArray()

    accuracy=(cm[0][0]+cm[1][1])/cm.sum()

    precision=(cm[0][0])/(cm[0][0]+cm[1][0])

    recall=(cm[0][0])/(cm[0][0]+cm[0][1])
    
    
    evaluator=BinaryClassificationEvaluator(rawPredictionCol = "rawPrediction",
                                            labelCol = label)
    
    AUC = evaluator.evaluate(predict_test)
    
    result = {"score" : accuracy, "precision" : precision, "recall" : recall, "AUC" : AUC}
    
    return result

In [48]:
compute_model(lr, "flg_cmd_lowcostIndex", sample_data)

{'score': 0.9662769784172662,
 'precision': 0.9641148325358851,
 'recall': 1.0,
 'AUC': 0.903076210716278}

#### Predire alors les clients lowcoast sur un sample de data n'ayant pas servi à l'apprentissage

In [49]:
# We predict on our test data
predict_test = model.transform(test)

In [50]:
# We print prediction and real value of "flg_cmd_lowcostIndex"
predict_test.select("flg_cmd_lowcostIndex","prediction").show(10)

+--------------------+----------+
|flg_cmd_lowcostIndex|prediction|
+--------------------+----------+
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 1.0|       1.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
|                 1.0|       1.0|
|                 0.0|       0.0|
|                 0.0|       0.0|
+--------------------+----------+
only showing top 10 rows



##### Random Forest

In [51]:
classifier = RandomForestClassifier(labelCol="flg_cmd_lowcostIndex", 
                                    featuresCol="indexedFeatures",
                                    maxDepth=15, numTrees=100)


In [52]:
model_rf = classifier.fit(train)

evaluer les performance de notre modele

In [100]:
# une autre technique pour evaluer le modele

from pyspark.ml.evaluation import BinaryClassificationEvaluator


evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

We put all in the function compute_model

In [53]:
compute_model(classifier, "flg_cmd_lowcostIndex", sample_data)

{'score': 0.966726618705036,
 'precision': 0.964576352321685,
 'recall': 1.0,
 'AUC': 0.937663694539755}

### DATA FRAME PART

In [ ]:
from pyspark.ml.linalg import Vectors
scoreAndLabels = map(lambda x: (Vectors.dense([1.0 - x[0], x[0]]), x[1]),
                     [(0.1, 0.0), (0.1, 1.0), (0.4, 0.0), (0.6, 0.0), (0.6, 1.0), (0.6, 1.0), (0.8, 1.0)])
dataset = spark_session.createDataFrame(scoreAndLabels, ["raw", "label"])
dataset.head()

#### how to create a spark data frame

In [79]:
from pyspark.sql import Row
import datetime

l = [(datetime.date(2018,1,3), 'Ankit',25, 'F'),
     (datetime.date(2018,2,3), 'Jalfaizy',22,'F'),
     (datetime.date(2018,1,5), 'saurabh',20,'F'),
     (datetime.date(2018,1,12), 'Bala',26,'F'),
     (datetime.date(2018,7,9), 'Jules',19,'F') ,
     (datetime.date(2018,3,18), 'Arild',43,'M'),
     (datetime.date(2018,1,5), 'sarah',20,'M'),
     (datetime.date(2018,8,12), 'Boly',33,'M'),
     (datetime.date(2018,4,6), 'Anita',35,'M'),
     (datetime.date(2018,12,6), 'Jules',22,'M'),
     (datetime.date(2018,7,24), 'Soul',20,'F'),
     (datetime.date(2018,6,17), 'Gral',54,'F'),
     (datetime.date(2018,9,7), 'Apoh',18,'F'),
     (datetime.date(2018,10,4), 'Dony',32,'F'),
     (datetime.date(2018,2,5), 'Tanoh',31,'F'),
     (datetime.date(2018,11,12), 'Issouf',27,'M'),
     (datetime.date(2018,10,3), 'Bilé',29,'M'),
     (datetime.date(2018,5,3), 'Gagnon',20,'M'),
     (datetime.date(2018,3,5), 'Papiss',28,'M'),
     (datetime.date(2018,2,12), 'Kravitz',34,'M'),
     (datetime.date(2018,5,9), 'Mouli',35,'F'),
     (datetime.date(2018,8,3), 'Jacques',27,'F'),
     (datetime.date(2018,12,5), 'soum',22,'F'),
     (datetime.date(2018,4,12), 'MBra',36,'F')]

rdd = spark_session.sparkContext.parallelize(l)
people = rdd.map(lambda x: Row(date=x[0], name=x[1], age=int(x[2]), sex = x[3]))
schemaPeople = spark_session.createDataFrame(people)
schemaPeople.toPandas()

,age,date,name,sex
0,25,2018-01-03,Ankit,F
1,22,2018-02-03,Jalfaizy,F
2,20,2018-01-05,saurabh,F
3,26,2018-01-12,Bala,F
4,19,2018-07-09,Jules,F
5,43,2018-03-18,Arild,M
6,20,2018-01-05,sarah,M
7,33,2018-08-12,Boly,M
8,35,2018-04-06,Anita,M
9,22,2018-12-06,Jules,M


1- compter le nombre de personne totale 
2- compter le nombre de fille et de garcon
3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)
4 - quelle est la date de dernière visite de chaque client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)
5 - quels sont les personnes qui ont une date de visite < 400 jours.

In [81]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

,age,date,name,sex,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,F,2019-11-02,668
1,22,2018-02-03,Jalfaizy,F,2019-11-02,637
2,20,2018-01-05,saurabh,F,2019-11-02,666
3,26,2018-01-12,Bala,F,2019-11-02,659
4,19,2018-07-09,Jules,F,2019-11-02,481
5,43,2018-03-18,Arild,M,2019-11-02,594
6,20,2018-01-05,sarah,M,2019-11-02,666
7,33,2018-08-12,Boly,M,2019-11-02,447
8,35,2018-04-06,Anita,M,2019-11-02,575
9,22,2018-12-06,Jules,M,2019-11-02,331


####  1- compter le nombre de personne totale

In [82]:
# Compter le nombre total de personnes
schemaPeople.count()

24

#### 2- compter le nombre de fille et de garcon

In [83]:
# Count the number of personne by sex
schemaPeople.groupBy("sex").count().show()

+---+-----+
|sex|count|
+---+-----+
|  F|   14|
|  M|   10|
+---+-----+



#### 3- quel est l'age moyen, median mini et maxi dans chaque groupe (garcon, fille)

In [84]:
from pyspark.sql.functions import mean, min, max
schemaPeople.select([mean('age'), min('age'), max('age')]).show()

+--------+--------+--------+
|avg(age)|min(age)|max(age)|
+--------+--------+--------+
|   28.25|      18|      54|
+--------+--------+--------+



In [85]:
schemaPeople.groupby('sex').agg(f.mean(schemaPeople.age).alias('avg'),
    f.min(schemaPeople.age).alias('min'),
    f.max(schemaPeople.age).alias('max'),
    f.count(schemaPeople.age).alias('count')).show()


+---+------------------+---+---+-----+
|sex|               avg|min|max|count|
+---+------------------+---+---+-----+
|  F|27.642857142857142| 18| 54|   14|
|  M|              29.1| 20| 43|   10|
+---+------------------+---+---+-----+



In [86]:
schemaPeople.groupby('sex').agg({'age': 'mean', 'age': 'min'}).show()

+---+--------+
|sex|min(age)|
+---+--------+
|  F|      18|
|  M|      20|
+---+--------+



In [87]:
# Average age by sex 
schemaPeople.groupBy('sex').mean().show()

+---+------------------+
|sex|          avg(age)|
+---+------------------+
|  F|27.642857142857142|
|  M|              29.1|
+---+------------------+



In [88]:
# Max age by sex 
schemaPeople.groupBy('sex').max().show()

+---+--------+
|sex|max(age)|
+---+--------+
|  F|      54|
|  M|      43|
+---+--------+



In [89]:
# Min age by se
schemaPeople.groupBy('sex').min().show()

+---+--------+
|sex|min(age)|
+---+--------+
|  F|      18|
|  M|      20|
+---+--------+



In [91]:
# First quartil , Median ,Thrid quartil  age by sex

schemaPeople.approxQuantile("age", [0.25,0.5, 0.75], 0.000001)
#schemaPeople.groupBy("sex").show()

[20.0, 27.0, 33.0]

#### 4 - quelle est la date de dernière visite de chaque 
#### client par rapport à la date d'aujourd'hui (la colonne date correspond à la date de visite)


In [92]:
dd = schemaPeople.select("*",
                         f.lit(datetime.date.today()).alias("date_max"))
dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit')).toPandas()

,age,date,name,sex,date_max,days_since_last_visit
0,25,2018-01-03,Ankit,F,2019-11-02,668
1,22,2018-02-03,Jalfaizy,F,2019-11-02,637
2,20,2018-01-05,saurabh,F,2019-11-02,666
3,26,2018-01-12,Bala,F,2019-11-02,659
4,19,2018-07-09,Jules,F,2019-11-02,481
5,43,2018-03-18,Arild,M,2019-11-02,594
6,20,2018-01-05,sarah,M,2019-11-02,666
7,33,2018-08-12,Boly,M,2019-11-02,447
8,35,2018-04-06,Anita,M,2019-11-02,575
9,22,2018-12-06,Jules,M,2019-11-02,331


In [93]:
# Question 5
jj = dd.select("*", f.datediff('date_max', 'date')\
                    .alias('days_since_last_visit'))

In [94]:
jj.filter(jj['days_since_last_visit'] < 400).count()

5

In [95]:
# Verifier s'il y a des valeurs manquantes
# Rajouter une colonne mois correspondant au mois de chaque individu dans le data frame
# Créer une colonne adulte qui prendra la valeur 0 pour les personnes de moins de 25 ans et 1 dans le cas contraire


#### Verifier s'il y a des valeurs manquantes

In [96]:
# Count number of missiong values
from pyspark.sql.functions import col,sum
# We can count the missing values by summing the boolean output of the isNull() method

check_missing_values(schemaPeople)

,age,date,name,sex
0,0,0,0,0


####  Créer une colonne adulte qui prendra 
#### la valeur 0 pour les personnes de moins de 25 ans et 1 dans le cas contraire


In [97]:
# Créer une colonne adulte qui prendra 
#la valeur 0 pour les personnes de moins de 25 ans et 1 dans le cas contraire

import pyspark.sql.functions as F
schemaPeople = schemaPeople.withColumn('adulte',
    F.when((schemaPeople.age < 25), 0).otherwise(1))

In [98]:
schemaPeople.show(2)

+---+----------+--------+---+------+
|age|      date|    name|sex|adulte|
+---+----------+--------+---+------+
| 25|2018-01-03|   Ankit|  F|     1|
| 22|2018-02-03|Jalfaizy|  F|     0|
+---+----------+--------+---+------+
only showing top 2 rows



#### Rajouter une colonne mois correspondant au mois de chaque individu dans le data frame

In [99]:
# Add a new column corresponding to each month of every individual

def add_column_month(df) :

    df = df.withColumn('mois',
        F.month(df.date))
    return df

In [100]:
add_column_month(schemaPeople).toPandas().iloc[0:2,:]

,age,date,name,sex,adulte,mois
0,25,2018-01-03,Ankit,F,1,1
1,22,2018-02-03,Jalfaizy,F,0,2


In [101]:
schemaPeople.groupby('adulte').count().toPandas()

,adulte,count
0,1,15
1,0,9


In [ ]:
pip install scikit-learn

     |██████▎                         | 1.4MB 1.2kB/s eta 1:17:32